In [ ]:
import glob
import keras
import random
import os.path
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from random import sample 
from shutil import copyfile
from numpy.random import seed
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.preprocessing.image import load_img
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Flatten, Dense, BatchNormalization, Lambda, Dropout

In [ ]:
print("tf version", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
print("Using default strategy for CPU and single GPU")
strategy = tf.distribute.get_strategy()

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

## Utility functions

In [ ]:
# ensure consistency across runs
seed(1)
tf.random.set_seed(1)

is_kaggle = True
path_to_images = '../input/siim-isic-melanoma-classification/jpeg/train'
path_to_csv = "../input/siim-isic-melanoma-classification/train.csv"

if not os.path.isfile(path_to_csv):
    is_kaggle = False
    path_to_images = 'input'
    path_to_csv = "input/train.csv"

In [ ]:
def get_data(path_to_csv):
    df = pd.read_csv(path_to_csv)
    return df
        


def check_for_missing_and_null(df):
    null_df = pd.DataFrame({
                            'percent_null': df.isnull().sum() * 100 / len(df),
                            'percent_zero': df.isin([0]).sum() * 100 / len(df)
                            })
    return null_df


def display_image(path):
    img = mpimg.imread(path)
    imgplot = plt.imshow(img)
    plt.show()

def rgb2gray(rgb):
    # source: https://stackoverflow.com/questions/12201577/how-can-i-convert-an-rgb-image-into-grayscale-in-python
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def standardize_image(imageData):
    # Find the mean and std dev intensity values of the image, and standerdize it
    mean_intensity = np.mean(imageData)
    std_intensity = np.std(imageData)
    new_img = imageData.copy()
    new_img = (new_img - mean_intensity)/std_intensity
    return new_img

def image_distribution(path):
    ## helper function to print the image and the intensity distribution
    image = mpimg.imread(path)
    image = rgb2gray(image)  

    f = plt.figure()
    f.set_figwidth(10)
    
    # standardize the image data
    image = standardize_image(image)
    
    s1 = f.add_subplot(1, 2, 1)
    s1.set_title('Image')
    plt.imshow(image, cmap='gray')
    
    s2 = f.add_subplot(1, 2, 2)
    s2.set_title('Intensity Distribution')
    plt.hist(image.ravel(), bins = 256)
    
    plt.show()


In [ ]:
df = get_data(path_to_csv)
df.head()

- The diagnosis feature is a simple diagnosis of the cancer.
- The benign_malignant feature is a feature that determines whether the tumor is benign or malignant (benign is harmless, malignant is harmful)
- The anatom_site_general_challenge tells us where the cancer is.
- The target is the target feature.

In [ ]:
for image_path in glob.glob(path_to_images + '/*.jpg')[:5]:
    display_image(image_path)

## Data Exploration and Preprocessing
Source: https://towardsai.net/p/data-analysis/exploratory-data-analysis-in-python-ebdf643a33f6

1. Identification of variables and data types
2. Analyzing the basic metrics
3. Non-Graphical Analysis
4. Graphical Analysis
5. Missing value treatment

### 1. Identification of variables and data types

In [ ]:
df.dtypes

We have two numerical variables (age_approx and target) and six categorial variables (image_name, patient_id, sex and anatom_site_general_challenge, diagnosis, benign_malignant)

### 2. Analyzing the basic metrics

In [ ]:
print("Number of rows in the dataset:", df.shape[0])
print("Number of columns in the dataset:", df.shape[1])

In [ ]:
df.describe()

The column age consits of 33.126 data points. The mean patient age is 49, the min age is 0 and the max age is 90. We will ignore the target columns in for now.

### 3. Non-Graphical Analysis

In [ ]:
df.patient_id.nunique()

In [ ]:
df.patient_id.value_counts()[:10]

Interestingly we only have 2.056 unique patients. From some patients we have have up to 115 images.
Maybe we have to consider this when we later check for Bias in our model. -> Why?

### 4. Graphical Analysis

In [ ]:
custom_bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df.plot.hist(df.age_approx, bins=custom_bins)
plt.show()


In [ ]:
age_10_or_younger = df[df['age_approx'] < 11]
len(age_10_or_younger)

We have 19 images of patients age 10 or younger.

In [ ]:
ax = df.anatom_site_general_challenge.value_counts().plot(kind='bar')
ax.set(ylabel = 'Anatom Site')



We can see that skin lesions most often occur at the torso, followed by the lower extremity.

In [ ]:
ax = df.sex.value_counts().plot(kind='bar')
ax.set(ylabel = 'Anatom Site')

print("Male patients", len(df[df['sex']=="male"]))
print("Female patients", len(df[df['sex']=="female"]))

There are 1.099 more images of male patients than female patients in the dataset.

In [ ]:
male_df = df[df['sex']=="male"]
female_df = df[df['sex']=="female"]

In [ ]:
ax = male_df.anatom_site_general_challenge.value_counts().plot(kind='bar')
ax.set(ylabel = 'Anatom Site')

In [ ]:
ax = female_df.anatom_site_general_challenge.value_counts().plot(kind='bar')
ax.set(ylabel = 'Anatom Site')

When looking at the body parts of the images and the genders we can see no differcence.

In [ ]:
for image_path in glob.glob(path_to_images + '/*.jpg')[:5]:
    image_distribution(image_path)

The intensity distribution of the images seem to follow normal distribution. 

### 5. Missing value treatment

In [ ]:
check_for_missing_and_null(df)

In [ ]:
print("Number of rows without data:", df.anatom_site_general_challenge.isnull().sum())

Because there are only 351 rows withou a value, I will remove these data points, because then we will have a clean df. 

In [ ]:
df = df.dropna() 
check_for_missing_and_null(df)

In [ ]:
df.shape

After the data cleaning we have a dataset with eight colunmns and 32.531 rows. Now let's take a deeper look at the data.

In [ ]:
df.head(1)

In [ ]:
benign_df = df[df.benign_malignant == "benign"]
malignant_df = df[df.benign_malignant == "malignant"]

In [ ]:
print("Rows benign_df", benign_df.shape[0])
print("Rows malignant_df", malignant_df.shape[0])
print("%", (malignant_df.shape[0] / benign_df.shape[0]) * 100)

We can see that arround 1.8 percent of all skin lesions are malignant.

In [ ]:
ax = df.diagnosis.value_counts().plot(kind='bar')
ax.set(ylabel = 'Diagnosis')

In [ ]:
df.diagnosis.value_counts()

We have seven types of diagnosed cancerous growths here:

- Unkown: a possibly novel type of growth
- Nevus: (from Google) a usually non-cancerous disorder of pigment-producing skin cells commonly called birth marks or moles.
- Melanoma: Skin cancer's form (what we are working with)
- Seborrheic keratosis: Brown, waxy and patchy growths that are not related to skin cancer.
- Lentigo NOS: A type of skin cancer that starts from the outside of the skin and attacks by going inword.
- Lichenoid keratosis: It is a thin pigmented sort of plaque, if you will.
- Solar lentigo: Like lentigo but caused by UV rays from the sun (very common in Delhi)
- cafe-au-lait macule: French for "coffee with milk". These are brownish spots also called "giraffe spots".
- atypical melanocytic proliferation: Abnormal quantities of melanin appear on the skin.

Unkown is with almost 80% the most ocurring value

## Prepare dataset

In [ ]:
df.head(1)

In [ ]:
df["new image_path"] = ""
for index, row in df.iterrows():
    df.at[index,'image_path'] = path_to_images + "/" + row['image_name'] + ".jpg"

del df['image_name']

In [ ]:
df.head(1)

## Get balanced dataset


In [ ]:
# 20 % in test set
# stratify makes sure that in your train set the specific class is balanced [if in the whole DF there are 40% of subjects with this class, then the train set will also have 40 %]
train_df, test_df = train_test_split(df, test_size = 0.2, stratify = df['target'])

In [ ]:
print("train", train_df.shape)
print("test", test_df.shape)

In [ ]:
ax = test_df.target.value_counts().plot(kind='bar')
ax.set(ylabel = 'Diagnosis')

For better training we need to have a balanced test df

In [ ]:
p_inds = test_df[test_df.target==1].index.tolist()
np_inds = test_df[test_df.target==0].index.tolist()
np_sample = sample(np_inds,len(p_inds))
test_df = test_df.loc[p_inds + np_sample]

In [ ]:
ax = test_df.target.value_counts().plot(kind='bar')
ax.set(ylabel = 'Diagnosis')

In [ ]:
test_df.shape

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 64


def image_augmentation():
    """Helper function to create ImageDataGenerator function
    Returns: image data generator function
    """
    idg = ImageDataGenerator(rescale=1 / 255.0,
                            horizontal_flip=True,
                            vertical_flip=True,
                            height_shift_range=0.1,
                            width_shift_range=0.1,
                            rotation_range=25,
                            shear_range=0.1,
                            zoom_range=0.15)
    return idg


def make_train_gen(df):
    """Generate batches of tensor image data with real-time data augmentation.
    The data will be looped over (in batches).
    Args:
        df ([dataframe]): [dataframe of the training data]
    Returns:
        [generator function]: Generator function for training data
    """
    idg = image_augmentation()
    train_gen = idg.flow_from_dataframe(dataframe=df,
                                        directory=None,
                                        x_col='image_path',
                                        y_col='target',
                                        class_mode='raw',
                                        target_size=IMG_SIZE,
                                        batch_size=BATCH_SIZE
                                        )
    return train_gen


def make_test_gen(df):
    """Generate batches of tensor image data with real-time data augmentation.
    The data will be looped over (in batches).
    Args:
        df ([dataframe]): [dataframe of the testing data]
    Returns:
        [generator function]: Generator function for testing data
    """
    test_idg = ImageDataGenerator(rescale=1. / 255.0)
    test_gen = test_idg.flow_from_dataframe(dataframe=df,
                                            directory=None,
                                            x_col='image_path',
                                            y_col='target',
                                            class_mode='raw',
                                            shuffle=False,
                                            target_size=IMG_SIZE,
                                            batch_size=BATCH_SIZE)
    return test_gen


def create_train_data(train_df):
    train_gen = make_train_gen(train_df)
    return train_gen


def create_test_data(test_df):
    train_gen = make_test_gen(test_df)
    return train_gen

In [ ]:
test_gen = create_test_data(test_df)
train_gen = create_train_data(train_df)

## Model

We will use ResNet, because it has the best accuracy and a good training time. If you want to read more about it, here is a great article: https://towardsdatascience.com/the-w3h-of-alexnet-vggnet-resnet-and-inception-7baaaecccc96

For a guid to Transfer Learning with the ResNet50 visit: https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b

In [ ]:
input_shape = (224, 224, 3)

In [ ]:
# The option include_top=False allows feature extraction by removing the last dense layers. This let us control the output and input of the model.
res_model = ResNet50(weights='imagenet', include_top=False, input_tensor=tf.keras.Input(shape=input_shape))

In [ ]:
print("Number of layers:", len(res_model.layers))

for layer in res_model.layers[:143]:
    layer.trainable = False

for layer in res_model.layers[138:]:
    print(layer.name, '-', layer.trainable)

In [ ]:
# New Sequentail Model
model = Sequential()

# Add the convolutional part of the VGG16 model 
model.add(res_model)

# Flatten the output of the res_model model because it is from a convolutional layer.
model.add(Flatten())
## ?????
model.add(BatchNormalization())
# Add a dense (aka. fully-connected) layer. This is for combining features that the res_model model has recognized in the image.
model.add(Dense(256, activation='relu'))
# Add a dropout-layer which may prevent overfitting and improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.5))

model.add(Flatten())
## ?????
model.add(BatchNormalization())
# Add a dense (aka. fully-connected) layer. This is for combining features that the res_model model has recognized in the image.
model.add(Dense(128, activation='relu'))
# Add a dropout-layer which may prevent overfitting and improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.5))

model.add(Flatten())
## ?????
model.add(BatchNormalization())
# Add a dense (aka. fully-connected) layer. This is for combining features that the res_model model has recognized in the image.
model.add(Dense(64, activation='relu'))
# Add a dropout-layer which may prevent overfitting and improve generalization ability to unseen data e.g. the test-set.
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
EPOCHS = 30
LEARNING_RATE = 1e-4

In [ ]:
## Set our optimizer, loss function, and learning rate
optimizer = Adam(lr=LEARNING_RATE)
loss = 'binary_crossentropy'
metrics=['binary_crossentropy', 'accuracy']

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
def save_history(history):
    """Helper function to save a png image of the loss and accuracy
    Args:
        history ([tf history]): The history object of a tf model
    """
    f = plt.figure()
    f.set_figwidth(15)

    f.add_subplot(1, 2, 1)
    plt.plot(history.history['val_loss'], label='val loss')
    plt.plot(history.history['loss'], label='train loss')
    plt.legend()
    plt.title("Modell Loss")

    f.add_subplot(1, 2, 2)
    plt.plot(history.history['val_accuracy'], label='val accuracy')
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.legend()
    plt.title("Modell Accuracy")

    plt.savefig(DIRECTORY_ROOT + '/model/history.png')

In [ ]:
weight_path = "model.hdf5"
checkpoint = ModelCheckpoint(weight_path,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto',
                            save_weights_only=True)

early = EarlyStopping(monitor='val_loss',
                    mode='auto',
                    patience=10)

callbacks_list = [checkpoint, early]

In [ ]:
testX, testY = test_gen.next() 

In [ ]:
history = model.fit(train_gen, validation_data=(testX, testY), epochs=EPOCHS, callbacks=callbacks_list, verbose=1)

In [ ]:
save_history(history)

In [ ]:
# save model architecture to a .json:
model_json = model.to_json()
with open(DIRECTORY_ROOT + "my_model.json", "w") as json_file:
    json_file.write(model_json)